In [ ]:
# steps
# 1. split df into num and cat features
# 2. fill NaN values
# 3. impute missing values (num)
# 4. create scaler (num)
# 5. make num_transformer pipe
# 6. impute missing values (cat)
# 7. create encorder (cat)
# 8. make cat_transformer pipe
# 9. make column transformer (combine num/cat transformers)
# 10. add model to final pipe

# Libraries 

In [43]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate

# Data Set

In [3]:
# Shape, 
# dtypes,
# duplicates
# missing values count
# look at the missing values

test_df = pd.read_csv("data/test.csv", index_col='Id')
train_df = pd.read_csv('data/train.csv',index_col="Id")


In [4]:
train_df.shape

(1460, 80)

In [50]:
y_train = train_df['SalePrice']
train_df = train_df.drop(columns='SalePrice')

In [5]:
train_df.drop_duplicates(inplace=True)

In [14]:

missing_values_later = list(train_df.isna().sum().sort_values(ascending=False).head(19).index)

In [16]:
train_df.drop(columns=missing_values_later, inplace=True)

In [18]:
train_df.isna().sum().sum()

0

# Preprocessing Features

In [21]:
# define the features and target
train_df.dtypes.value_counts()

int64     34
object    27
dtype: int64

## Numerical features

In [22]:
train_num = train_df.select_dtypes(include='int64')

In [23]:
train_num

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,8450,7,5,2003,2003,706,0,150,856,...,0,61,0,0,0,0,0,2,2008,208500
2,20,9600,6,8,1976,1976,978,0,284,1262,...,298,0,0,0,0,0,0,5,2007,181500
3,60,11250,7,5,2001,2002,486,0,434,920,...,0,42,0,0,0,0,0,9,2008,223500
4,70,9550,7,5,1915,1970,216,0,540,756,...,0,35,272,0,0,0,0,2,2006,140000
5,60,14260,8,5,2000,2000,655,0,490,1145,...,192,84,0,0,0,0,0,12,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,7917,6,5,1999,2000,0,0,953,953,...,0,40,0,0,0,0,0,8,2007,175000
1457,20,13175,6,6,1978,1988,790,163,589,1542,...,349,0,0,0,0,0,0,2,2010,210000
1458,70,9042,7,9,1941,2006,275,0,877,1152,...,0,60,0,0,0,0,2500,5,2010,266500


In [27]:
# scale this
pipe_num = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
     ('scalier', RobustScaler())
]).set_output(transform='pandas')
pipe_num

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scalier', RobustScaler())])

In [28]:
pipe_num.fit_transform(train_num)

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,0.2,-0.254076,0.5,0.0,0.652174,0.243243,0.452790,0.0,-0.559829,-0.269652,...,0.000000,0.529412,0.0,0.0,0.0,0.0,0.0,-1.333333,0.0,0.541506
2,-0.6,0.030015,0.0,3.0,0.065217,-0.486486,0.834679,0.0,-0.330769,0.538308,...,1.773810,-0.367647,0.0,0.0,0.0,0.0,0.0,-0.333333,-0.5,0.220173
3,0.2,0.437624,0.5,0.0,0.608696,0.216216,0.143910,0.0,-0.074359,-0.142289,...,0.000000,0.250000,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.720024
4,0.4,0.017663,0.5,0.0,-1.260870,-0.648649,-0.235170,0.0,0.106838,-0.468657,...,0.000000,0.147059,272.0,0.0,0.0,0.0,0.0,-1.333333,-1.0,-0.273728
5,0.2,1.181201,1.0,0.0,0.586957,0.162162,0.381186,0.0,0.021368,0.305473,...,1.142857,0.867647,0.0,0.0,0.0,0.0,0.0,2.000000,0.0,1.035406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,0.2,-0.385746,0.0,0.0,0.565217,0.162162,-0.538435,0.0,0.812821,-0.076617,...,0.000000,0.220588,0.0,0.0,0.0,0.0,0.0,0.666667,-0.5,0.142815
1457,-0.6,0.913167,0.0,1.0,0.108696,-0.162162,0.570727,163.0,0.190598,1.095522,...,2.077381,-0.367647,0.0,0.0,0.0,0.0,0.0,-1.333333,1.0,0.559357
1458,0.4,-0.107831,0.5,4.0,-0.695652,0.324324,-0.152334,0.0,0.682906,0.319403,...,0.000000,0.514706,0.0,0.0,0.0,0.0,2500.0,-0.333333,1.0,1.231776


## Categorical Features

In [29]:
train_cat = train_df.select_dtypes(include='object')

In [30]:
train_cat

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,ExterCond,Foundation,Heating,HeatingQC,CentralAir,KitchenQual,Functional,PavedDrive,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
2,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,...,TA,CBlock,GasA,Ex,Y,TA,Typ,Y,WD,Normal
3,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
4,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,...,TA,BrkTil,GasA,Gd,Y,Gd,Typ,Y,WD,Abnorml
5,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,...,TA,PConc,GasA,Ex,Y,TA,Typ,Y,WD,Normal
1457,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,...,TA,CBlock,GasA,TA,Y,TA,Min1,Y,WD,Normal
1458,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,...,Gd,Stone,GasA,Ex,Y,Gd,Typ,Y,WD,Normal


In [34]:
pipe_cat= Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
     ('endcoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore',drop='if_binary' ))
]).set_output(transform='pandas')
pipe_cat

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('endcoder',
                 OneHotEncoder(drop='if_binary', handle_unknown='ignore',
                               sparse_output=False))])

In [35]:
pipe_cat.fit_transform(train_cat)

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1457,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1458,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Preprocessor

In [51]:
preprocessor = ColumnTransformer([
    ('pipe_cat', pipe_cat,make_column_selector(dtype_include='object')),
    ('pipe_num', pipe_num,make_column_selector(dtype_include='int64')),
]).set_output(transform='pandas')
preprocessor

ColumnTransformer(transformers=[('pipe_cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('endcoder',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fa8961fa410>),
                                ('pipe_num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scalier', RobustScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fa8961fa380>)])

In [52]:
train_df_preprocessed = preprocessor.fit_transform(train_df)

In [53]:
train_df_preprocessed.shape

(1460, 212)

# Modeling

In [44]:
y_train = train_df['SalePrice']


## Linear Regression 


In [54]:
model = LinearRegression()
cv_result = cross_validate(model,train_df_preprocessed,y_train)

In [55]:
cv_result

{'fit_time': array([0.04615784, 0.05071425, 0.03689742, 0.19145036, 0.0387392 ]),
 'score_time': array([0.01208615, 0.01152611, 0.02080703, 0.01278305, 0.01120162]),
 'test_score': array([-1.95108411e+12, -4.32616589e+13, -9.94506027e+11, -7.39390130e+11,
        -1.12999528e+14])}

In [59]:
model.fit(train_df_preprocessed,y_train)

LinearRegression()

# Base Prediction


In [56]:
preprocessed_X_test = preprocessor.transform(test_df)

In [57]:
preprocessed_X_test

,pipe_cat__MSZoning_C (all),pipe_cat__MSZoning_FV,pipe_cat__MSZoning_RH,pipe_cat__MSZoning_RL,pipe_cat__MSZoning_RM,pipe_cat__Street_Pave,pipe_cat__LotShape_IR1,pipe_cat__LotShape_IR2,pipe_cat__LotShape_IR3,pipe_cat__LotShape_Reg,...,pipe_num__GarageArea,pipe_num__WoodDeckSF,pipe_num__OpenPorchSF,pipe_num__EnclosedPorch,pipe_num__3SsnPorch,pipe_num__ScreenPorch,pipe_num__PoolArea,pipe_num__MiscVal,pipe_num__MoSold,pipe_num__YrSold
Id,,,,,,,,,,,,,,,,,,,,,
1461,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.035197,0.833333,-0.367647,0.0,0.0,120.0,0.0,0.0,0.000000,1.0
1462,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,-0.695652,2.339286,0.161765,0.0,0.0,0.0,0.0,12500.0,0.000000,1.0
1463,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.008282,1.261905,0.132353,0.0,0.0,0.0,0.0,0.0,-1.000000,1.0
1464,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,-0.041408,2.142857,0.161765,0.0,0.0,0.0,0.0,0.0,0.000000,1.0
1465,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.107660,0.000000,0.838235,0.0,0.0,144.0,0.0,0.0,-1.666667,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,-1.987578,0.000000,-0.367647,0.0,0.0,0.0,0.0,0.0,0.000000,-1.0
2916,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,-0.803313,0.000000,-0.014706,0.0,0.0,0.0,0.0,0.0,-0.666667,-1.0
2917,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.397516,2.821429,-0.367647,0.0,0.0,0.0,0.0,0.0,1.000000,-1.0


In [60]:
predictions = model.predict(preprocessed_X_test)

In [62]:
preprocessed_X_test['SalePrice'] = predictions

## KNNregessor

In [63]:
preprocessed_X_test.head()

,pipe_cat__MSZoning_C (all),pipe_cat__MSZoning_FV,pipe_cat__MSZoning_RH,pipe_cat__MSZoning_RL,pipe_cat__MSZoning_RM,pipe_cat__Street_Pave,pipe_cat__LotShape_IR1,pipe_cat__LotShape_IR2,pipe_cat__LotShape_IR3,pipe_cat__LotShape_Reg,...,pipe_num__WoodDeckSF,pipe_num__OpenPorchSF,pipe_num__EnclosedPorch,pipe_num__3SsnPorch,pipe_num__ScreenPorch,pipe_num__PoolArea,pipe_num__MiscVal,pipe_num__MoSold,pipe_num__YrSold,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1461,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.833333,-0.367647,0.0,0.0,120.0,0.0,0.0,0.000000,1.0,118376.187500
1462,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,2.339286,0.161765,0.0,0.0,0.0,0.0,12500.0,0.000000,1.0,160929.468750
1463,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.261905,0.132353,0.0,0.0,0.0,0.0,0.0,-1.000000,1.0,187633.953125
1464,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,2.142857,0.161765,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,195317.140625
1465,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.000000,0.838235,0.0,0.0,144.0,0.0,0.0,-1.666667,1.0,220458.718750


# Results

In [64]:
output = preprocessed_X_test[['SalePrice']].reset_index()

In [65]:
output

,Id,SalePrice
0,1461,118376.187500
1,1462,160929.468750
2,1463,187633.953125
3,1464,195317.140625
4,1465,220458.718750
...,...,...
1454,2915,70903.250000
1455,2916,71697.843750
1456,2917,171762.734375
1457,2918,100435.203125


In [66]:
output.to_csv('submission.csv', index=False)